<!--NOTEBOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="./figures/LogoOpenclassrooms.png">
<font size="4">
    
Cette étude a été réalisée dans le cadre du 6ème projet de ma formation Datascientist dispensée en MOOC par 

<font color='blus'>Openclassrooms / écoles Centrale-Supélec</font>.




<p></p><p></p><p></p>





**Le problème posé :**


*Stack Overflow est un site célèbre de question-réponses liées au développement informatique. Pour poser une question sur ce site, il faut entrer plusieurs tags de manière à retrouver facilement la question par la suite. Pour les utilisateurs expérimentés cela ne pose pas de problème, mais pour les nouveaux utilisateurs, il serait judicieux de suggérer quelques tags relatifs à la question posée.*

*Amateur de Stack Overflow, qui vous a souvent sauvé la mise, vous décidez d'aider la communauté en retour. Pour cela, vous développez un système de suggestion de tags pour le site. Celui-ci prendra la forme d’un algorithme de machine learning qui assigne automatiquement plusieurs tags pertinents à une question.*



**Solutions mises en oeuvre**

Les solutions de suggestion de tags présentés ici se basent sur des modèles supervisés de machine learning.

Les modèmes mis en oeuvre : 
    * Naive Multinomial Baysien

Ce notebook utilise les données issues des notebooks : 


**P6_DadaAnalysis.ipynb**

**P6_UnsupervizedMethods.ipynb**



In [8]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# 4. Supervized methods

**Loading TF-IDF operator and CSR matrix**

In [9]:
import p5_util
file_name="./data/csr_matrix_tdif_ngram_2_2.dump"
csr_matrix = p5_util.object_load(file_name)
print(csr_matrix.shape)

file_name="./data/vectorizer_tdif_ngram_2_2.dump"
vectorizer = p5_util.object_load(file_name)

p5_util.object_load : fileName= ./data/csr_matrix_tdif_ngram_2_2.dump
(24604, 898)
p5_util.object_load : fileName= ./data/vectorizer_tdif_ngram_2_2.dump


## 4.1. Test dataset vectorization

In [10]:
import pandas as pd
file_name="./data/test_StackOverFlow_BodyTitleTags.csv"
df_sof_body_test=pd.read_csv(file_name)

**Test dataset standardization : applies on `Body` column**

In [11]:
import p6_util
ser_sof_body_test = p6_util.p6_df_standardization(df_sof_body_test.Body)


Cleaning text in-between markers <code></code> markers...

Cleaning LXML markers...

Remove verbs from sentences...

Filtering alpha-numeric words from sentences...

Removing stopwords...

Lemmatization ...


In [12]:
csr_matrix_test = vectorizer.transform(ser_sof_body_test)

In [13]:
csr_matrix_test.A.shape, df_sof_body_test.shape

((16359, 898), (16359, 5))

## 4.2. Target processing : list of TAGS are encoded

### 4.2.1. Target data are loaded and formated

In [14]:
import pandas as pd
file_name="./data/train_StackOverFlow_BodyTitleTags.csv"
ser_sof_tags=pd.read_csv(file_name).Tags.copy()

In [15]:
ser_sof_tags.shape

(24604,)

**Tags are converted into a lists of words.**

In [16]:
import p6_util
ser_sof_listtags = ser_sof_tags.apply(p6_util.clean_marker_text, leading_marker='<', trailing_marker='>')

**A unique list of all TAGs is built**

In [19]:
import p6_util
list_all_tags = p6_util.p6_get_list_all_tag(ser_sof_listtags)

**For each row, TAGs are encoded**

In [20]:
list_list_encoded_row = p6_util.p6_encode_target(list_all_tags, ser_sof_listtags.tolist())

**Simple test for checking some random encoded rows**

In [21]:
%reload_ext autoreload
import p6_util
row=3000
print(p6_util.p6_get_list_tag_from_encoded_row(list_list_encoded_row,row,list_all_tags))
print(ser_sof_listtags[row])

['64bit' 'macos' 'python']
['python', 'macos', '64bit']


In [22]:
len(list_list_encoded_row)

24604

In [23]:
arr = np.array(list_list_encoded_row)

In [24]:
import numpy as np
from scipy import sparse

csr_matrix_encoded_target=sparse.csr_matrix(np.array(list_list_encoded_row))

## 4.3. Applying Naive Bayes classifier

In [27]:
csr_matrix.A.shape,csr_matrix_encoded_target.shape

((24604, 898), (24604, 6436))

one versus Rest leads having one classifier per class.

`OneVsRestClassifier` classifier is used because of multiple classes.

In [28]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
import numpy as np

model = OneVsRestClassifier(MultinomialNB()).fit(csr_matrix, csr_matrix_encoded_target)
#model = MultinomialNB().fit(X, y)
#model.fit(csr_matrix.A,df)

In [29]:
csr_matrix_test.shape

(16359, 898)

In [30]:
prediction=model.predict(csr_matrix_test)

In [36]:
print(prediction.shape)
#np.where(prediction[0]>0)
index_array_row = np.array(prediction[0]>0)

(16359, 6436)


In [39]:
index_array_row

array(<1x6436 sparse matrix of type '<class 'numpy.bool_'>'
	with 0 stored elements in Compressed Sparse Row format>, dtype=object)

In [31]:
model.score(csr_matrix_test,csr_matrix_encoded_target)

ValueError: Found input variables with inconsistent numbers of samples: [24604, 16359]

In [ ]:
csr_matrix_test.A.shape

In [42]:
print(accuracy(model, csr_matrix_test))

NameError: name 'accuracy' is not defined